In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
movies = pd.read_csv("movies.dat", sep="::", encoding = "latin-1")
ratings = pd.read_csv("ratings.dat", sep="::",  encoding = "latin-1")
users = pd.read_csv("users.dat", sep="::", encoding = "latin-1")

train_set = pd.read_csv("u2.base", delimiter='\t')
test_set = pd.read_csv("u2.test", delimiter='\t')

# Conveting to numpy array
train_set = np.array(train_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')

# Getting the number of users and movies
no_users = int(max(max(train_set[:, 0]), max(test_set[:,0])))
no_movies = int(max(max(train_set[:, 1]), max(test_set[:,1])))
print("Number of users:", no_users)
print("Number of movies:", no_movies)

<ipython-input-3-e7d04131710c>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv("movies.dat", sep="::", encoding = "latin-1")
<ipython-input-3-e7d04131710c>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("ratings.dat", sep="::",  encoding = "latin-1")


Number of users: 943
Number of movies: 1682


<ipython-input-3-e7d04131710c>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv("users.dat", sep="::", encoding = "latin-1")


In [4]:
def convert(data):
  new_data = []
  for id in range(1, no_users +1):
    movie_id = data[:, 1][data[:, 0] == id]
    rating_id = data[:, 2][data[:, 0] == id]
    ratings = np.zeros(no_movies)
    ratings[movie_id -1] = rating_id
    new_data.append(list(ratings))
  return new_data

train_set = convert(train_set)
test_set = convert(test_set)

In [5]:
train_set = torch.FloatTensor(train_set)
test_set = torch.FloatTensor(test_set)

Creating our stack Autoencoders


In [6]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(no_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, no_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [7]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

Traing SAE

In [8]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(no_users):
        input = Variable(train_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = no_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.6516)
epoch: 2 loss: tensor(1.0970)
epoch: 3 loss: tensor(1.0541)
epoch: 4 loss: tensor(1.0393)
epoch: 5 loss: tensor(1.0314)
epoch: 6 loss: tensor(1.0267)
epoch: 7 loss: tensor(1.0240)
epoch: 8 loss: tensor(1.0219)
epoch: 9 loss: tensor(1.0206)
epoch: 10 loss: tensor(1.0198)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0183)
epoch: 13 loss: tensor(1.0178)
epoch: 14 loss: tensor(1.0177)
epoch: 15 loss: tensor(1.0168)
epoch: 16 loss: tensor(1.0167)
epoch: 17 loss: tensor(1.0163)
epoch: 18 loss: tensor(1.0167)
epoch: 19 loss: tensor(1.0159)
epoch: 20 loss: tensor(1.0160)
epoch: 21 loss: tensor(1.0159)
epoch: 22 loss: tensor(1.0159)
epoch: 23 loss: tensor(1.0154)
epoch: 24 loss: tensor(1.0154)
epoch: 25 loss: tensor(1.0147)
epoch: 26 loss: tensor(1.0128)
epoch: 27 loss: tensor(1.0113)
epoch: 28 loss: tensor(1.0095)
epoch: 29 loss: tensor(1.0085)
epoch: 30 loss: tensor(1.0052)
epoch: 31 loss: tensor(1.0042)
epoch: 32 loss: tensor(0.9997)
epoch: 33 loss: t

In [9]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(no_users):
    input = Variable(train_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[(target == 0).unsqueeze(0)] = 0
        loss = criterion(output, target)
        mean_corrector = no_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test loss: tensor(0.9550)
